In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')
train_data = pd.read_csv('/kaggle/input/titanic/train.csv')
print(train_data.isnull().sum())
X = train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'], axis=1)
y = train_data.Survived

In [ ]:
X = train_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived'], axis=1)
y = train_data.Survived
X = pd.get_dummies(X)
X = X.fillna({'Age': X.Age.median()})
print(X.isnull().sum())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
max_depth_values = range(1,100)
scores_data = pd.DataFrame()


for max_depth in max_depth_values:
    clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
    clf.fit(X_train,y_train)
    
    train_score = clf.score(X_train,y_train)
    test_score = clf.score(X_test,y_test)
   
    mean_cross_val_score = cross_val_score(clf, X_train, y_train, cv=5).mean()
    
    temp_data_frame = pd.DataFrame({'max_depth':[max_depth], 
                                    'train_score':[train_score], 
                                    'test_score':[test_score],
                                   'cross_val_score':[mean_cross_val_score]})
    scores_data = scores_data.append(temp_data_frame)

    
scores_data.head()

In [ ]:
import seaborn as sns
scores_data_long = pd.melt(scores_data, id_vars=['max_depth'], 
                           value_vars=['train_score', 'test_score', 'cross_val_score'],\
                           var_name='set_type', value_name='score')
sns.lineplot(data=scores_data_long, x="max_depth", y="score", hue="set_type")

In [ ]:
best_clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=11)
best_clf.fit(X_train, y_train)
best_clf.score(X_test, y_test)

In [ ]:
test_data.head()

In [ ]:
X_test_data = test_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
X_test_data = X_test_data.fillna({'Age': X.Age.median()})
X_test_data = X_test_data.fillna({'Fare': X.Fare.median()})
X_test_data = pd.get_dummies(X_test_data)

In [ ]:
Survived = pd.DataFrame(best_clf.predict(X_test_data), columns=['Survived'])
PassengerId = test_data.PassengerId
result = pd.concat([PassengerId, Survived], axis=1)


result.to_csv('submission.csv', index=False)

In [ ]:
result.head()

In [ ]:
result = pd.DataFrame({'PassengerId':[PassengerId], 
                                    'Survived':[Survived]})